In [1]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objs as go
import plotly.plotly as ply
import plotly
from plotly.offline import plot, init_notebook_mode, iplot

%matplotlib inline

# Defining repository

In [2]:
userhome = os.path.expanduser('~')
repository = userhome + r'/historage-mining/historage/hbase/'

# Opening the earliest affected version

In [3]:
affected_version = pd.read_csv('/Users/YusufNugroho/historage-mining/Analyze/analyze_hbase/step_3/affected_version/earliest_version.csv')
cols = ['bug_id','affected_version','date_release']
aff = affected_version[cols]
aff

,bug_id,affected_version,date_release
0,HBASE-9426,0.94.11RC0,2013-08-13
1,HBASE-5209,0.90.5RC0,2011-12-09
2,HBASE-15467,2.0.0-alpha-1RC0,2017-06-07
3,HBASE-13686,1.1.0-SNAPSHOT-testing,2015-04-29
4,HBASE-10598,0.94.16RC0,2014-01-10
5,HBASE-10569,None,NaN


In [4]:
#Defining main folder of the files
directory = userhome + r'/historage-mining/Analyze/analyze_hbase/'

In [5]:
filedata = pd.read_csv('diffbugs_file_differentvalue_withfilepath.csv')
filedata = filedata[['bug_ref','filename']]

details = filedata.join(aff.set_index('bug_id')[['affected_version','date_release']], on='bug_ref')
details

,bug_ref,filename,affected_version,date_release
0,HBASE-10569,/Users/YusufNugroho/historage-mining/historage...,None,NaN
1,HBASE-13686,/Users/YusufNugroho/historage-mining/historage...,1.1.0-SNAPSHOT-testing,2015-04-29
2,HBASE-13686,/Users/YusufNugroho/historage-mining/historage...,1.1.0-SNAPSHOT-testing,2015-04-29
3,HBASE-10569,/Users/YusufNugroho/historage-mining/historage...,None,NaN
4,HBASE-10569,/Users/YusufNugroho/historage-mining/historage...,None,NaN


# Open diff file affected by HBASE-13686 (RateLimiter & TestRateLimiter.java)

In [6]:
blamefiles = ['blame_RateLimiter.java.csv', 'blame_TestRateLimiter.java.csv']
diff_files = [['RateLimiter.java-HBASE-13686_histogrambuglines.diff', 
               'RateLimiter.java-HBASE-13686_minimalbuglines.diff',
               'RateLimiter.java-HBASE-13686_myersbuglines.diff',
               'RateLimiter.java-HBASE-13686_patiencebuglines.diff'],
              ['TestRateLimiter.java-HBASE-13686_histogrambuglines.diff',
               'TestRateLimiter.java-HBASE-13686_minimalbuglines.diff',
               'TestRateLimiter.java-HBASE-13686_myersbuglines.diff',
               'TestRateLimiter.java-HBASE-13686_patiencebuglines.diff']
             ]

In [7]:
bug_id = re.compile('(HBASE-[0-9]*)')
bug_id = bug_id.findall(diff_files[0][0])
bug_id[0]

'HBASE-13686'

In [8]:
for m, dfiles in enumerate(diff_files):
    for n, df in enumerate(dfiles):
        print (df)

RateLimiter.java-HBASE-13686_histogrambuglines.diff
RateLimiter.java-HBASE-13686_minimalbuglines.diff
RateLimiter.java-HBASE-13686_myersbuglines.diff
RateLimiter.java-HBASE-13686_patiencebuglines.diff
TestRateLimiter.java-HBASE-13686_histogrambuglines.diff
TestRateLimiter.java-HBASE-13686_minimalbuglines.diff
TestRateLimiter.java-HBASE-13686_myersbuglines.diff
TestRateLimiter.java-HBASE-13686_patiencebuglines.diff


In [9]:
#Open csv file from result of git blame for <filename>.java
blames = []
for num, blame in enumerate(blamefiles):
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(directory + r"step_3/" + blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-13686_<diffalgorithm>changes.diff vs blame_<filename>
    for m, df in enumerate(diff_files[num]):
        diff_file = open (directory + r"file-diff/buggylines/" + df,'r')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            for x in range(num, len(origin_code)-1):
                if bugs in org_code:
                    codex = origin_code[x][1]
                    if bugs == codex:
                        temp = [origin_code[x][0], bugs, "True", origin_code[x][2]]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
                else:
                    temp = ['NA', bugs, "False", 'NA']
                    blame_compare.append(temp)
                    break;
                        
        with open (directory + r'step_3/diff-file_blame-file_comparison/diff_' + df + '_' + blame, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)